# WES|WGSPipeline 

In [ ]:
import os
import sys

#s3 address of input files and output files
s3_input_files_address = "s3://path/to/input"
s3_output_files_address = "s3://path/to/output"

## CFNCluster name
your_cluster_name = ""

## The private key pair for accessing cluster.
private_key = "/path/to/key.pem"

## Project information
project_name = ""

#bwa_gatk or bwa_mutect
workflow = ""

#hg19 or GRCm38
genome = ""

## If delete cfncluster after job is done.
delete_cfncluster= False

print("variables set")

In [ ]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

In [ ]:
from util import PipelineManager
from util import DesignFileLoader

## Possible analysis_steps inputs for the two workflows. Order of input does not matter.

##bwa_gatk: "fastqc", "trim", "bwa", "sort", "dedup", "split", "postalignment", 
#"haplotype", "merge", "combine_vcf", "filter"

##bwa_mutect: "fastqc", "trim" , "bwa", "multiqc", "sort", "dedup", "split", "mutect", "bam_merge", "pair_vcf_merge"
analysis_steps = {
                    #"fastqc"
                    #,"trim"
                    #,"bwa"
                    #,"multiqc"
                    #,"sort"
                    #,"dedup"
                    #,"split"
                    #,"postalignment"
                    #,"haplotype"
                    #,"mutect"
                    #,"merge" 
                    #,"bam_merge"
                    #,"pair_vcf_merge"
                    #,"combine_vcf"
                    #"filter"
                }

#add design file path here
#examples in cirrus_root/data/cirrus-ngs/
design_file = "/path/to/design_file.txt"

sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("DNASeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, "NA", pairs_list)